In [1]:
!pip install sqlalchemy psycopg2-binary pandas xgboost scikit-learn
!pip install pymysql


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary librariess
from sqlalchemy import create_engine, text
import pandas as pd

# Database configuration
db_user = "postgres"
db_pass = "admin123"
db_host = "localhost"  # Use public IP if testing from remote/Colab
db_port = "5432"
db_name = "postgres"

# Construct the full database URL
DATABASE_URL = f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"

# Create engine
engine = create_engine(DATABASE_URL)


In [3]:
# Test the connection with a simple query
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1;"))  # Wrap SQL with text()
        print("Database connection successful:", result.fetchone())
except Exception as e:
    print("Connection failed:", e)


Database connection successful: (1,)


In [10]:
# Query to get product and invoice data
query = """
SELECT
    p.product_id,
    i.invoice_date,
    p.quantity,
    pd.product_name
FROM
    product p
INNER JOIN
    invoices i ON i.id = p.invoice_id
INNER JOIN
    purchase_detail pd on p.product_id = pd.id
"""
# Fetch data into a DataFrame
df = pd.read_sql(query, engine)

# Display the first few rows of the data
df.head()


,product_id,invoice_date,quantity,product_name
0,1152,2024-01-01,1.0,Allbirds - Dhoti
1,763,2024-01-01,2.0,Off-White - Nehru Jacket
2,1106,2024-01-01,1.0,Allbirds - Parkas
3,753,2024-01-01,1.0,Puma - Gloves
4,1105,2024-01-01,1.0,Puma - Windbreakers


In [15]:
# Convert invoice_date to datetime
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

# Extract year and month
df['year'] = df['invoice_date'].dt.year
df['month'] = df['invoice_date'].dt.month

# Combine into a date column for sorting and indexing
df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))

# Sort data by product_id and date
df.sort_values(by=['product_id', 'date'], inplace=True)

# Display the updated DataFrame
df.tail()


,product_id,invoice_date,quantity,product_name,year,month,date
1507,1152,2025-05-25,1.0,Allbirds - Dhoti,2025,5,2025-05-01
1512,1152,2025-05-25,1.0,Allbirds - Dhoti,2025,5,2025-05-01
1519,1152,2025-05-27,1.0,Allbirds - Dhoti,2025,5,2025-05-01
1524,1152,2025-05-27,1.0,Allbirds - Dhoti,2025,5,2025-05-01
1525,1152,2025-05-29,1.0,Allbirds - Dhoti,2025,5,2025-05-01


In [16]:
# Group by product_id and predict sales for each product
product_data =df
unique_products = product_data['product_id'].unique()

# Dictionary to store predictions for each product
predictions = {}

# Loop through each product and train a model for it
for product_id in unique_products:
    product_data_filtered = product_data[product_data['product_id'] == product_id]
    
    if product_data_filtered.empty or len(product_data_filtered) < 3:
        predictions[product_id] = {"message": f"Not enough data to predict for product_id {product_id}"}
        continue
    
    # Resample monthly and sum quantities
    product_data_filtered = product_data_filtered.set_index('invoice_date').resample('ME').sum(numeric_only=True).fillna(0)

    # Prepare features
    product_data_filtered['month'] = product_data_filtered.index.month
    product_data_filtered['year'] = product_data_filtered.index.year
    product_data_filtered['time_index'] = range(len(product_data_filtered))

    # Features and target
    X = product_data_filtered[['month', 'year', 'time_index']]
    y = product_data_filtered['quantity']

    # Train-test split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Train model
    from xgboost import XGBRegressor
    model = XGBRegressor(n_estimators=100)
    model.fit(X_train, y_train)

    # Predict next month
    last_date = product_data_filtered.index.max()
    next_date = last_date + pd.DateOffset(months=1)

    next_input = pd.DataFrame([{
        'month': next_date.month,
        'year': next_date.year,
        'time_index': len(product_data_filtered)
    }])

    prediction = model.predict(next_input)[0]

    predictions[product_id] = {
        "predicted_month": next_date.strftime("%B %Y"),
        "predicted_quantity": round(float(prediction))
    }

product_name_map = product_data[['product_id', 'product_name']].drop_duplicates().set_index('product_id').to_dict()['product_name']


# Find the product with the highest predicted sales
highest_sales_product_id = max(predictions, key=lambda x: predictions[x]["predicted_quantity"])


# Get the product name for the highest predicted sales
highest_sales_product_name = product_name_map.get(highest_sales_product_id, "Unknown Product")


# Output the result
highest_sales_prediction = predictions[highest_sales_product_id]

print(f"Product '{highest_sales_product_name}' (ID: {highest_sales_product_id}) is predicted to have the highest sales next month.")
print(f"Predicted sales for {highest_sales_prediction['predicted_month']}: {highest_sales_prediction['predicted_quantity']}")


Product 'Off-White - Nehru Jacket' (ID: 763) is predicted to have the highest sales next month.
Predicted sales for June 2025: 27
